# Topic Modeling 

In this notebook we wiil texplain what we first understand by topic modelling, what are the most used methods for topic modelling and I will conclude with dynamic topic modeling.

### Topic Modelling Definition

### Topic Modelling Methods

### Dynamic Topic Modelling

In standard topic modelling we assume the order of document does not matter and therfore this approach is not suitable for time-stamped data. 
Dynamic topic modelling on the other end approach to track how languages and topic changes in a time-stamped corpus